In [46]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
import numpy as np
from psycopg2 import sql

### Опис 
СУБД: PostgreSQL 16

Сервер: BM

БД: postgres

Скрипт підключається до БД, створює таблицю bm_subsriptions, проходиться по файлам у папці та додає до таблиці.

In [27]:
conn = psycopg2.connect(database = "postgres", 
                        user = "postgres", 
                        host= 'localhost',
                        password = "2100",
                        port = 5432)


In [28]:
cur = conn.cursor()

In [26]:
cur.execute("""CREATE TABLE bm_subscriptions (
                event_date DATE,
                app_name VARCHAR(255),
                app_apple_id BIGINT,
                subscription_name VARCHAR(255),
                subscription_apple_id BIGINT,
                subscription_group_id BIGINT,
                subscription_duration VARCHAR(50),
                introductory_price_type VARCHAR(50),
                introductory_price_duration VARCHAR(50),
                marketing_opt_in_duration VARCHAR(50),
                customer_price DECIMAL(10, 2),
                customer_currency VARCHAR(10),
                developer_proceeds DECIMAL(10, 2),
                proceeds_currency VARCHAR(10),
                preserved_pricing BOOLEAN,
                proceeds_reason TEXT,
                client VARCHAR(50),
                device VARCHAR(50),
                country VARCHAR(10),
                subscriber_id BIGINT,
                subscriber_id_reset BOOLEAN,
                refund BOOLEAN,
                purchase_date DATE,
                units INTEGER);""")

conn.commit()

In [48]:
def clean_boolean(value):
    if pd.isna(value) or value == '' or value == ' ':
        return None
    return bool(value)

def clean_date(value):
    if pd.isna(value) or value == '' or value == ' ':
        return None
    try:
        return pd.to_datetime(value).date()
    except:
        return None

def clean_float(value):
    if pd.isna(value) or value == '' or value == ' ':
        return None
    try:
        return float(value)
    except:
        return None

def clean_int(value):
    if pd.isna(value) or value == '' or value == ' ':
        return None
    try:
        return int(value)
    except:
        return None

In [38]:
folder_path = "/Users/kostas/Documents/Tableau/BM_test_task/bm_files"

In [49]:
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        
        try:
            df = pd.read_csv(file_path, delimiter="\t")
            
            date_fields = ['Event Date', 'Purchase Date']
            boolean_fields = ['Preserved Pricing', 'Subscriber ID Reset', 'Refund']
            float_fields = ['Customer Price', 'Developer Proceeds']
            int_fields = ['App Apple ID', 'Subscription Apple ID', 'Subscription Group ID', 'Units']

            for field in date_fields:
                if field in df.columns:
                    df[field] = df[field].apply(clean_date)
            
            for field in boolean_fields:
                if field in df.columns:
                    df[field] = df[field].apply(clean_boolean)
            
            for field in float_fields:
                if field in df.columns:
                    df[field] = df[field].apply(clean_float)
            
            for field in int_fields:
                if field in df.columns:
                    df[field] = df[field].apply(clean_int)

            for index, row in df.iterrows():
                cur.execute(sql.SQL("""
                    INSERT INTO bm_subscriptions (
                        event_date, app_name, app_apple_id, subscription_name, 
                        subscription_apple_id, subscription_group_id, subscription_duration, 
                        introductory_price_type, introductory_price_duration, marketing_opt_in_duration, 
                        customer_price, customer_currency, developer_proceeds, proceeds_currency, 
                        preserved_pricing, proceeds_reason, client, device, country, 
                        subscriber_id, subscriber_id_reset, refund, purchase_date, units
                    ) VALUES ({})
                """).format(sql.SQL(', ').join([sql.Placeholder()] * 24)), (
                    row['Event Date'], row['App Name'], row['App Apple ID'], row['Subscription Name'], 
                    row['Subscription Apple ID'], row['Subscription Group ID'], row['Subscription Duration'], 
                    row['Introductory Price Type'], row['Introductory Price Duration'], row['Marketing Opt-In Duration'], 
                    row['Customer Price'], row['Customer Currency'], row['Developer Proceeds'], row['Proceeds Currency'], 
                    row['Preserved Pricing'], row['Proceeds Reason'], row['Client'], row['Device'], row['Country'], 
                    row['Subscriber ID'], row['Subscriber ID Reset'], row['Refund'], row['Purchase Date'], row['Units']
                ))
            
            conn.commit()
            print(f"Successfully processed file: {filename}")
        
        except Exception as e:
            print(f"Error occurred for file {filename}: {e}")
            conn.rollback()

Successfully processed file: 20190205.txt
Successfully processed file: 20190204.txt
Successfully processed file: 20190210.txt
Successfully processed file: 20190206.txt
Successfully processed file: 20190207.txt
Successfully processed file: 20190203.txt
Successfully processed file: 20190202.txt
Successfully processed file: 20190201.txt
Successfully processed file: 20190209.txt
Successfully processed file: 20190208.txt


In [50]:
cur.close()
conn.close()